Aggregating data to the level of individuals

In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
import readVCF
from matplotlib import pyplot as plt
import pandas as pd
from bsm import clean
import statsmodels as sm

In [4]:
v1 = ['AF', 'ALT', 'BaseQRankSum', 'DP', 'FILTER/PASS', 'FS', 'GWASpval', 'REF', 'ReadPosRankSum', 'SOR', 'VQSLOD', 'chromatinState_DLPFC', 'culprit', 'evolConstrain', 'szdbCNVcount']
v2 = ['Dx', 'AntipsychAtyp', 'AntipsychTyp', 'Institution', 'EV.3']
calls = readVCF.readVCFs()
calls = clean.prettify_colnames(clean.preselect(calls, vnames=v1 + v2))
calls = clean.dummify_var(calls, vname='Dx')
calls = clean.impute_vars(calls, vnames=['ReadPosRankSum', 'EV_3'], v1=v1, v2=v2)
calls.count()

Welcome, Attila Jones!

ReadPosRankSum
EV_3


AF                      3301
ALT                     3301
BaseQRankSum            3301
DP                      3301
FILTER_PASS             3301
FS                      3301
GWASpval                3301
REF                     3301
ReadPosRankSum          3298
SOR                     3301
VQSLOD                  3301
chromatinState_DLPFC    3301
culprit                 3301
evolConstrain           3301
szdbCNVcount            3301
Dx                      3301
AntipsychAtyp           3301
AntipsychTyp            3301
Institution             3301
EV_3                    3301
dtype: int64